In [12]:
import sys
sys.path.insert(0, '..') #This line adds '..' to the path so we can import the net_framework python file
from net_framework import *
import numpy as np
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import scipy.stats as sp

The following cell allows you to toggle warnings on and off. It helps the readability of the results but warnings in general are important and should not be ignored. 

In [13]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

Hi! This notebook will serve as a tutorial to walk through the basics of this project. In this notebook, we will model the truthfulness of a class of mathematical statements that amount to the addition of two integers between -5 and 5. We need to limit the range of integers because it would take an infinite amount of compute power to fully conceptualize the addition of any two integers with no bounds. 

To model the information content of the chosen set of statements, we will go through the following procedure-

1. Defining a "sub-language" to constrain the set of statements. This will require going through the process of formalizing what the set of statements you want to model are and how to interpret them. 
2. Finding an efficient representation that can be inputted into neural networks for the characters in the language and the statements you want to model. For the most part, we will be using one-hot vectors for this task (https://en.wikipedia.org/wiki/One-hot). 
3. Generating training and validation datasets for our statements. These datasets should consist of input vectors that represent legal statements in the "sub-language" and the associated truth values for each of these statements. 
4. Use the data to train a set of neural networks with different structures neural complexities. The ones with training error below a certain threshold will be validated. We will be using a threshold value of 0.05 for this notebok. The networks with validation error below that threshold will be classified as "good" networks and the "good" network with minimal neural complexity will be used as the final model to represent the statements. 
5. (optional) Try finding the information complexity (the amount data it takes to train the network) of that network structure by training it with different amounts of training data and seeing how much training data is required for it to have training and validation errors below the threshold. 
6. Save the statistics from the final network. This will include things such as the final neural complexity (the number of hidden layer nodes in the network), information complexity (the amount data it takes to train the network), final validation/training error, final weights of the network and plots of the training error as the number of training cycles increases. 

## Step 1: Defining a Sub-Language

This is a purely conceptual step but is easily the most important and difficult part of this project. We need to define a language to constrain the set of statements we are working with. To do this, we need to define three things:

1. The characters in the language
2. The way these characters are legally allowed to be combined. 
3. How to determine the truthfulness of a given statement. 

In our case, the set of characters we are using can be split into two groups. 

1. The set of integers from -5 to 5, inclusive. We will call this set, $Z$.
2. The symbols '+' and '='.

These characters can only legally be combined in a single way.

Let $z_1$, $z_2$ and $z_3$ be three elements of $Z$. Then, we define a legal 'sentence' in this language as the following combination of characters: 

$$z_1 + z_2 = z_3$$.

And the veracity of these statements can be understood in the normal way, the statement is true if and only if the first two integers add up to the third. 

At first glance, it might seem unecessary to model the '+' and '=' symbols since they are in the same positions every time, but this is just a very simple example and modeling these types of symbols will be necessary when dealing with more complex syntaxes in the future. 

## Step 2: Finding a Representation

We now need to represent our language in a way that can be understood by the neural network. To do this, we will be using one-hot vectors (https://en.wikipedia.org/wiki/One-hot) to represent each character in the network. Each vector will have a dimensionality of 13 to represent each of the 11 integers as well the '+' and '=' symbols. 

We will use the following mapping: 

'+' maps to [1,0,0,...,0,0]  
'=' maps to [0,1,0,...,0,0]  
'-5' maps to [0,0,1,...,0,0]  
...  
'4' maps to [0,0,0,...,1,0]  
and  
'5' maps to [0,0,0,...,0,1]  

Each sentence in the langauge can be thought of as a sequence of five of these vectors stacked end to end. 

Srinath Edit: Since we are adding subtraction, we need a total vector size of 14.

## Step 3: Generating Training/Validation Data

Now, we have to write functions that will generate training or validation datasets on command. 

In [14]:
def one_hot(symbols):
    '''
    Converts symbol ('+', '=', int. between -50 to 50) array into 'stacked' one-hot vectors as specified above. 
    '''
    vector_stack = []
    for symbol in symbols:
        vector = np.zeros(14)
        if symbol == '+':
            vector[0] = 1
        elif symbol == '-':
            vector[1] = 1
        elif symbol == '=':
            vector[2] = 1
        else:
            idx = int(symbol) + 8
            vector[idx] = 1
        vector_stack = np.concatenate((vector_stack, vector))
    return np.asarray(vector_stack)
    
    
def gen_data(num_examples, randomize = True):
    
    '''
    Generates statements in this language as well as their veracity.
    
    Params
    ------
    num_examples : int
        The number of examples in the dataset. 
    randomize : bool
        Whether or not to randomize the output dataset. If
        False, returns the entire possible dataset.
        
Srinath Edit: Assuming we want to keep the data set the same size, we want to alternate between addition and subtraction. True is add and False is subtraction.

    Returns
    -------
    X : 2D numpy array
        Matrix of inputs where each row is a vector that represents a single sentence.
    Y : 1D numpy array
        Each element 1 if the corresponding statement in X is true and 0 otherwise. 
    '''
    X = []
    Y = []
    sentences = []
    i = 0
    add = True
    if randomize:
        while i < num_examples:
            if i < num_examples/2:
                #Randomly Choosing three integers between -50 and 50
                z = np.random.randint(-5, 5 + 1, 3)
                if add:
                    sentence = np.array([z[0], '+', z[1], '=', z[2]])
                    sentences.append(sentence)
                    X.append(one_hot(sentence))
                    if z[0] + z[1] == z[2]:
                        Y.append(1)
                    else:
                        Y.append(0)
                else:
                    sentence = np.array([z[0], '-', z[1], '=', z[2]])
                    sentences.append(sentence)
                    X.append(one_hot(sentence))
                    if z[0] - z[1] == z[2]:
                        Y.append(1)
                    else:
                        Y.append(0)
                add = not add
            else:
                #Choosing values such that the output is true to ensure that our training dataset
                #is not skewed with False results.
                z = np.random.randint(-5, 5 + 1, 2)
                z = np.append(z, z[0] + z[1]) if add else np.append(z, z[0] - z[1])
                #Ensuring the sentence is legal
                if z[2] in np.arange(-5, 6, 1):
                    if add:
                        sentence = np.array([z[0], '+', z[1], '=', z[2]])
                        sentences.append(sentence)
                        X.append(one_hot(sentence))
                        if z[0] + z[1] == z[2]:
                            Y.append(1)
                        else:
                            Y.append(0)
                    else:
                        sentence = np.array([z[0], '-', z[1], '=', z[2]])
                        sentences.append(sentence)
                        X.append(one_hot(sentence))
                        if z[0] - z[1] == z[2]:
                            Y.append(1)
                        else:
                            Y.append(0)
                    add = not add
                else:
                    i -= 1
            i += 1
            

        return np.asarray(X), np.asarray(Y).reshape(len(Y), 1), np.asarray(sentences)
    else:
        for i in np.arange(-5, 6):
            for j in np.arange(-5, 6):
                for k in np.arange(-5, 6):
                    z = [i,j,k]
                    sentence = np.array([z[0],'+', z[1], '=', z[2]])
                    sentences.append(sentence)
                    X.append(one_hot(sentence))
                    if z[0] + z[1] == z[2]:
                        Y.append(1)
                    else:
                        Y.append(0)
                    sentence = np.array([z[0], '-', z[1], '=', z[2]])
                    sentences.append(sentence)
                    X.append(one_hot(sentence))
                    if z[0] - z[1] == z[2]:
                        Y.append(1)
                    else:
                        Y.append(0)
        return np.asarray(X), np.asarray(Y).reshape(len(Y), 1), np.asarray(sentences)

In [31]:
data = gen_data(151, randomize = True)

We can now visualize our training data

In [32]:
X_train = data[0]
Y_train = data[1]
sentences = data[2]
example = 4
print('A typical sentence is: ' + str(sentences[example]))
print('The truth value of this sentence is: ' + str(Y_train[example]))
print('Its one-hot representation is: ' + str(X_train[example]))

A typical sentence is: ['1' '+' '3' '=' '0']
The truth value of this sentence is: [0]
Its one-hot representation is: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


## Step 4: Training and Validating the Networks

Now, we need to define and train a bunch of different neural networks of various shapes. We will train over 100 iterations with the entire dataset of $11^3 = 1331$ training samples. 

The neural networks we are using will have an input size of 5 * 13 = 65 since that is the size of 5 one-hot vectors stacked on each other. The output size will be 1. We will be trying networks with one hidden layer and varying sizes of 20, 10 and 5, networks with two hidden layers with sizes of [10,5] and [5,3]. We will also be using a learning rate of 10 but other learning rates can work as well. 

In this case, we are creating a binary network so we take the sigmoid of the output and round network values to 1 if they are greater than 0.5 and 0 otherwise. We want network which makes 0 errors which means it has learned the entire system. The loss function we are using during training is the binary cross entropy loss (don't worry about it for now), and the training error we are printing is the l1 norm, or the function $|y_{true} - y_{pred}|$

### Training

In [39]:
#Number of training iterations
num_iters = 200
#Size of training dataset
num_examples = 1331
#Listing out the shapes of each model
network_shapes = [(70, [5], 1), (70, [4], 1), (70, [3], 1), (70, [2], 1), (70, [2, 1], 1)]
#Learning rate of the network
rate = 7
#Generating Training Data
training_data = gen_data(num_examples, randomize = True)
X = torch.from_numpy(training_data[0]).float()
Y = torch.from_numpy(training_data[1]).float()

#Array of losses over training period for each network
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    

    for i in range(num_iters):
        #Calculating l1 error
        error = NN.l1error(Y, NN(X))
        if i == 0: 
            dh = display("#" + str(i) + " Error: " + str(error), display_id=True)
        else:
            dh.update("#" + str(i) + " Error: " + str(error))
            
        NN.train(X, Y)
    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num));

Network Shape:
Input Size = 70
Hidden Size = [5]
Output Size = 1


'#0 Error: 0.5116022825241089'

Network Shape:
Input Size = 70
Hidden Size = [4]
Output Size = 1


'#0 Error: 0.5085005164146423'

Network Shape:
Input Size = 70
Hidden Size = [3]
Output Size = 1


'#0 Error: 0.4911904036998749'

Network Shape:
Input Size = 70
Hidden Size = [2]
Output Size = 1


'#0 Error: 0.4982266426086426'

Network Shape:
Input Size = 70
Hidden Size = [2, 1]
Output Size = 1


'#0 Error: 0.5043677091598511'

### Validation

In this step, we will see how well the networks did at actually guessing the truth values of the statements and whether or not any of them got near 0 error. 0 error is possible if we round all network outputs to 0 or 1. 

In [40]:
num_examples = 1331
#Generating validation data
validation_data = gen_data(num_examples, randomize = True)
X = validation_data[0]
Y = validation_data[1]
#Converting validation data into pytorch format
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    
    #Loading the network we trained in the prev. section
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    NN.load_state_dict(torch.load("saved_networks/Net " + str(net_num)))
    validation_error = NN.l1error(Y, torch.round(NN(X)))

    print("The validation error is: " + str(validation_error), flush = True)  

Network Shape:
Input Size = 70
Hidden Size = [5]
Output Size = 1
The validation error is: 0.10743801295757294
Network Shape:
Input Size = 70
Hidden Size = [4]
Output Size = 1
The validation error is: 0.1743050366640091
Network Shape:
Input Size = 70
Hidden Size = [3]
Output Size = 1
The validation error is: 0.22464312613010406
Network Shape:
Input Size = 70
Hidden Size = [2]
Output Size = 1
The validation error is: 0.26897069811820984
Network Shape:
Input Size = 70
Hidden Size = [2, 1]
Output Size = 1
The validation error is: 0.3155522048473358


We see that the network with 3 nodes is the smallest that was able to get to 0.0 error. However, the one with 5 nodes was not which suggests the optimization process is imprecise and, in general, will have to be run multiple times to get a good error. If you rerun the above cells, you might not get the same results I did. 

To confirm that our network works, we should testits output for various indices and ensure that it is correct. Here, we will do so for 20 randomly picked indices, at least 10 of which have a true output

In [41]:
indices_true = np.argwhere(Y.T[0] == 1)[0]
random_true_indices = np.random.choice(indices_true, size = 10)
indices = np.random.random_integers(0, 1331, 10)
indices = np.append(indices, random_true_indices)
#The sentences we are modelling from the previous dataset.
sentences = validation_data[2]
#Getting the best network from the previous validation set
NN = Neural_Network(inputSize = 14*5, outputSize = 1,
                        hiddenSize = [3] , learning_rate = 7)
NN.load_state_dict(torch.load("saved_networks/Net 2"))
#Predicting values for the dataset
predicted = NN(X, binary = True)
for index in indices:
    print("The statement is: " + str(sentences[index]))
    print("Its truth value is: " + str(predicted[index].item()))
    print("Actual Value: " + str(Y[index]))

The statement is: ['4' '+' '1' '=' '5']
Its truth value is: 1.0
Actual Value: tensor([1.])
The statement is: ['0' '+' '4' '=' '4']
Its truth value is: 1.0
Actual Value: tensor([1.])
The statement is: ['-2' '+' '0' '=' '-2']
Its truth value is: 1.0
Actual Value: tensor([1.])
The statement is: ['4' '-' '3' '=' '4']
Its truth value is: 0.0
Actual Value: tensor([0.])
The statement is: ['-1' '+' '-5' '=' '1']
Its truth value is: 1.0
Actual Value: tensor([0.])
The statement is: ['-2' '+' '2' '=' '-5']
Its truth value is: 0.0
Actual Value: tensor([0.])
The statement is: ['-1' '-' '1' '=' '4']
Its truth value is: 1.0
Actual Value: tensor([0.])
The statement is: ['-5' '-' '-4' '=' '-1']
Its truth value is: 1.0
Actual Value: tensor([1.])
The statement is: ['-1' '-' '4' '=' '-2']
Its truth value is: 0.0
Actual Value: tensor([0.])
The statement is: ['0' '+' '0' '=' '0']
Its truth value is: 1.0
Actual Value: tensor([1.])
The statement is: ['4' '-' '4' '=' '0']
Its truth value is: 1.0
Actual Value: 

We see that our minimum complexity network does seem to predict each truth value correctly. This is a good confirmation step to make sure the network is working properly and there is no error in your code. 

## Step 6: Save and Discuss Results

The networks in this notebook are saved in the saved_networks folder. It is important to save these since sometimes training can take a long time and might not produce your desired result every time. 

We see from these results that our minimal complexity network in this case has three nodes. This makes logical sense since it essentially equates to one node per integer being chosen. Next, it would be useful to incorporate other operations such as subtraction, multiplication and division.